<a href="https://colab.research.google.com/github/p-tech/wbs-dm-2026/blob/main/joins_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import sqlite3
import random
from datetime import datetime, timedelta

db_path = "shop.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# Enable foreign keys
#cur.execute("PRAGMA foreign_keys = ON;")

#conn = sqlite3.connect(":memory:")
conn.execute("PRAGMA foreign_keys = ON;")  # turn off first
cur = conn.cursor()

# Drop tables if they exist
cur.executescript("""
DROP TABLE IF EXISTS order_products;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS customers;
""")

conn.execute("PRAGMA foreign_keys = ON;")  # turn back on

# -------------------------------------------------
# CREATE DATABASE (in memory)
# -------------------------------------------------

conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# Enable foreign keys
cur.execute("PRAGMA foreign_keys = ON;")

# -------------------------------------------------
# CREATE TABLES
# -------------------------------------------------

cur.execute("""
CREATE TABLE customers (
    id INTEGER PRIMARY KEY,
    fname TEXT NOT NULL,
    lname TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
);
""")

cur.execute("""
CREATE TABLE products (
    productID INTEGER PRIMARY KEY,
    pname TEXT NOT NULL,
    pcost REAL NOT NULL,
    pstock INTEGER NOT NULL
);
""")

cur.execute("""
CREATE TABLE orders (
    orderID INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    total_cost REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);
""")

cur.execute("""
CREATE TABLE order_products (
    orderID INTEGER,
    productID INTEGER,
    quantity INTEGER NOT NULL,
    PRIMARY KEY (orderID, productID),
    FOREIGN KEY (orderID) REFERENCES orders(orderID),
    FOREIGN KEY (productID) REFERENCES products(productID)
);
""")

# -------------------------------------------------
# INSERT CUSTOMERS (50)
# -------------------------------------------------

first_names = [
"John","Emma","Liam","Olivia","Noah","Ava","James","Sophia",
"Lucas","Isabella","Mason","Mia","Ethan","Amelia","Logan",
"Harper","Alexander","Ella","Daniel","Grace","Henry","Chloe",
"Sebastian","Lily","Jack","Emily","Owen","Aria","Leo","Scarlett",
"Thomas","Layla","Jacob","Hannah","Michael","Zoe","Benjamin",
"Nora","William","Riley","Elijah","Aurora","David","Violet",
"Joseph","Penelope","Samuel","Luna","Matthew","Ellie"
]

last_names = [
"Smith","Jones","Brown","Taylor","Davies","Wilson","Evans",
"Thomas","Roberts","Johnson","Walker","Wright","Hall","Allen",
"Young","King","Scott","Green","Adams","Baker","Nelson","Hill",
"Campbell","Mitchell","Carter","Phillips","Parker","Turner",
"Collins","Edwards","Stewart","Morris","Rogers","Cook","Morgan",
"Bell","Murphy","Bailey","Rivera","Cooper","Richardson","Cox",
"Howard","Ward","Torres","Peterson","Gray","Ramirez","James","Watson"
]

for i in range(50):
    email = f"{first_names[i].lower()}.{last_names[i].lower()}@email.com"
    cur.execute("INSERT INTO customers VALUES (?,?,?,?)",
                (i+1, first_names[i], last_names[i], email))

# -------------------------------------------------
# INSERT PRODUCTS (40 electronics)
# -------------------------------------------------

products = [
("iPhone 15",999.99,25),("Samsung Galaxy S24",899.99,30),
("MacBook Air M3",1299.99,15),("Dell XPS 13",1199.99,10),
("iPad Pro",899.99,20),("Samsung Tablet S9",749.99,18),
("Sony WH-1000XM5",349.99,40),("Bose Earbuds",279.99,35),
("Apple Watch 9",399.99,22),("Fitbit Charge 6",179.99,50),
("PlayStation 5",479.99,12),("Xbox Series X",479.99,14),
("Nintendo Switch",309.99,25),("Canon EOS R10",899.99,8),
("GoPro Hero 12",399.99,16),("HP LaserJet",199.99,20),
("Logitech Mouse",99.99,60),("RGB Keyboard",129.99,45),
("Samsung 4K Monitor",399.99,19),("LG UltraWide",499.99,11),
("External SSD 1TB",129.99,55),("USB-C Hub",49.99,80),
("JBL Speaker",149.99,37),("Amazon Echo",59.99,70),
("Google Nest Hub",99.99,65),("Smart TV 55",699.99,9),
("Wireless Charger",39.99,90),("Drone 4K",599.99,7),
("VR Headset",399.99,10),("WiFi 6 Router",199.99,28),
("Power Bank",29.99,100),("Graphics Tablet",249.99,13),
("Raspberry Pi 5",79.99,40),("Smart Doorbell",149.99,21),
("Security Camera",129.99,33),("Gaming Chair",299.99,12),
("USB Microphone",89.99,44),("HD Webcam",69.99,38),
("Smart Thermostat",199.99,17),("Electric Scooter",499.99,6)
]

for i, p in enumerate(products):
    cur.execute("INSERT INTO products VALUES (?,?,?,?)",
                (i+1, p[0], p[1], p[2]))

# -------------------------------------------------
# INSERT ORDERS (150)
# -------------------------------------------------

start_date = datetime(2025,1,1)

for order_id in range(1,151):

    customer_id = random.randint(1,45)  # customers 46–50 have no orders
    order_date = start_date + timedelta(days=random.randint(0,180))

    # Insert order first (required for foreign key)
    cur.execute("""
        INSERT INTO orders VALUES (?,?,?,?)
    """,(order_id, customer_id,
         order_date.strftime("%Y-%m-%d"), 0))

    product_ids = random.sample(range(1,35), random.randint(1,5))
    total_cost = 0

    for pid in product_ids:
        quantity = random.randint(1,2)

        cur.execute("""
            INSERT INTO order_products VALUES (?,?,?)
        """,(order_id, pid, quantity))

        cur.execute("SELECT pcost FROM products WHERE productID=?",(pid,))
        price = cur.fetchone()[0]
        total_cost += price * quantity

    cur.execute("""
        UPDATE orders
        SET total_cost = ?
        WHERE orderID = ?
    """,(round(total_cost,2), order_id))

conn.commit()

print("Database created successfully.")


Database created successfully.


Construct a join to extract the First Name, Last Name and Order Date

In [11]:
for row in cur.execute("""
SELECT c.fname, c.lname, o.order_date
FROM customers AS c
JOIN orders AS o ON c.id = o.customer_id
LIMIT 10;
"""):

 print (row)

('Amelia', 'Allen', '2025-02-15')
('Isabella', 'Johnson', '2025-05-23')
('Joseph', 'Torres', '2025-03-01')
('Joseph', 'Torres', '2025-02-07')
('Mia', 'Wright', '2025-02-17')
('David', 'Howard', '2025-05-02')
('Michael', 'Morgan', '2025-05-01')
('Grace', 'Baker', '2025-01-15')
('Ethan', 'Hall', '2025-01-16')
('Emma', 'Jones', '2025-01-05')


Show me the customers and the products they purchased.

In [13]:
cur.execute("""
SELECT
 c.fname,
 c.lname,
 p.pname
FROM
 customers AS c
JOIN
 orders AS o ON c.id = o.customer_id
JOIN
 order_products AS op ON o.orderID = op.orderID
JOIN
 products AS p ON op.productID = p.productID
WHERE
 c.id <= 10
ORDER BY
 c.id, o.order_date, p.pname;

""")

results = cur.fetchall()

# Print headers
print(f"{'First Name':<15}{'Last Name':<15}{'Product Name':<30}")
print(f"{'='*15:<15}{'='*15:<15}{'='*30:<30}")

# Print results
for row in results:
    print(f"{row[0]:<15}{row[1]:<15}{row[2]:<30}")

First Name     Last Name      Product Name                  
John           Smith          GoPro Hero 12                 
John           Smith          HP LaserJet                   
John           Smith          Smart Doorbell                
John           Smith          Wireless Charger              
John           Smith          iPhone 15                     
John           Smith          Amazon Echo                   
John           Smith          JBL Speaker                   
John           Smith          Wireless Charger              
John           Smith          iPad Pro                      
John           Smith          Drone 4K                      
John           Smith          External SSD 1TB              
John           Smith          PlayStation 5                 
John           Smith          Samsung Tablet S9             
John           Smith          Canon EOS R10                 
John           Smith          Nintendo Switch               
John           Smith    

Customers with no orders

In [15]:
for row in cur.execute("""
SELECT
 c.fname,
 c.lname,
 c.email,
 o.orderID
FROM
 customers AS c
LEFT JOIN
 orders AS o ON c.id = o.customer_id
WHERE
 o.orderID IS NULL;
"""):
    print(row)

('Daniel', 'Adams', 'daniel.adams@email.com', None)
('Jacob', 'Rogers', 'jacob.rogers@email.com', None)
('Nora', 'Bailey', 'nora.bailey@email.com', None)
('Violet', 'Ward', 'violet.ward@email.com', None)
('Penelope', 'Peterson', 'penelope.peterson@email.com', None)
('Samuel', 'Gray', 'samuel.gray@email.com', None)
('Luna', 'Ramirez', 'luna.ramirez@email.com', None)
('Matthew', 'James', 'matthew.james@email.com', None)
('Ellie', 'Watson', 'ellie.watson@email.com', None)


Products that have never been purchased

In [16]:
for row in cur.execute("""
SELECT p.productID, p.pname
FROM products AS p
LEFT JOIN order_products AS op ON p.productID = op.productID
WHERE op.productID IS NULL;
"""):
    print(row)

(35, 'Security Camera')
(36, 'Gaming Chair')
(37, 'USB Microphone')
(38, 'HD Webcam')
(39, 'Smart Thermostat')
(40, 'Electric Scooter')
